In [5]:
!pip install langchain  "pyautogen[retrievechat]"  PyPDF2  faiss-cpu


  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl.metadata (541 bytes)
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ----------------------- -------------- 41.0/67.3 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 67.3/67.3 kB 731.0 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached grpcio-1.65.5-cp310-cp310-win_amd64.whl.metadata (3.4 kB)
     ---------------------------------------- 0.0/43.7 kB ? eta -:--:--
     ----------------

In [ ]:
import autogen
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
config_list = [
    {
        "model": "gpt-4-1106-preview",
        "api_key": "openai_api",
    }
]

llm_config_proxy = {
    "seed": 42,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "request_timeout": 600
}

In [ ]:
reader = PdfReader('/content/openchat.pdf')
corpus = ''.join([p.extract_text() for p in reader.pages if p.extract_text()])

splitter =  RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200,)
chunks = splitter.split_text(corpus)

embeddings = OpenAIEmbeddings(openai_api_key = openai_api)
vectors = FAISS.from_texts(chunks, embeddings)

In [ ]:
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0),
    vectors.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", 
    return_messages=True),
)

In [ ]:
llm_config_assistant = {
    "Seed" : 42,
    "temperature": 0,
        "functions": [
        {
            "name": "answer_PDF_question",
            "description": "Answer any PDF related questions",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask in relation to PDF",
                    }
                },
                "required": ["question"],
            },
            
        }
    ],
    "config_list": config_list,
    "timeout": 120,
}

In [ ]:
assistant = autogen.AssistantAgent(
            name="assistant",
            llm_config=llm_config_assistant,
            system_message="""You are a helpful assistant, Answer the question based on the context. 
                              Keep the answer accurate. Respond "Unsure about answer" if not sure about the answer."""
            
        )

In [ ]:
def answer_PDF_question(question):
    # Sử dụng qa để trả lời câu hỏi
    response = qa({"question": question})
    return response['answer']

In [ ]:
user_proxy = autogen.UserProxyAgent(  
            name="user_proxy",
            human_input_mode="NEVER", 
            max_consecutive_auto_reply=10,
            code_execution_config={"work_dir": "coding"},
            # llm_config_assistant = llm_config_assistant,
            # function_map={
            #     "answer_PDF_question": llm_config_assistant["functions"][0]["name"]
            # }
            function_map={
                "answer_PDF_question": answer_PDF_question
            }
        )

In [ ]:
user_proxy.initiate_chat(
    assistant,
    message="""
Write a Openchat word blog post titled why openchat better than GPT3 that uses the exact keyword OpenChat 
at least once every 100 words. The blog post should include an introduction, main body,
 and conclusion. The conclusion should invite readers to leave a comment. The main
 body should be split into at least 4 different subsections.
"""
)

user_proxy (to assistant):


Write a Openchat word blog post titled why openchat better than GPT3 that uses the exact keyword OpenChat 
at least once every 100 words. The blog post should include an introduction, main body,
 and conclusion. The conclusion should invite readers to leave a comment. The main
 body should be split into at least 4 different subsections.


--------------------------------------------------------------------------------
assistant (to user_proxy):

# Why OpenChat is Better Than GPT-3

## Introduction

In the ever-evolving landscape of artificial intelligence, OpenChat has emerged as a groundbreaking platform, offering a unique set of capabilities that set it apart from its predecessors like GPT-3. In this blog post, we will delve into the reasons why OpenChat is not just a step ahead but a leap forward in AI communication technology.

## Main Body

### Enhanced Contextual Understanding

OpenChat's ability to understand context surpasses that of GPT-3. It can maintain the thread of a conversation over a longer period, which allows for more coherent and meaningful interactions. This is particularly beneficial in customer service applications where conversations can be complex and require a deep understanding of the issue at hand.

### Superior Customization

One of the key advantages of OpenChat is its superior customization options. Unlike GPT-3, OpenChat can be tailored to fit the specific needs of any business or application. This means that it can adhere to brand voice, manage specialized knowledge bases, and integrate seamlessly with existing systems, providing a more personalized experience for users.

### Advanced Learning Capabilities

OpenChat is designed to learn and adapt more efficiently than GPT-3. It can quickly incorporate new information and adjust its responses accordingly. This continuous learning process ensures that OpenChat remains up-to-date with the latest data, trends, and user preferences, making it an invaluable tool for dynamic and fast-paced environments.

### Open-Source Community

The open-source nature of OpenChat is a game-changer. It allows developers from around the world to contribute to its development, leading to rapid innovation and improvement. This collaborative approach ensures that OpenChat is constantly evolving and benefiting from the collective expertise of a global community, unlike the more closed ecosystem of GPT-3.

## Conclusion

OpenChat represents a significant advancement in AI-powered communication, offering enhanced contextual understanding, superior customization, advanced learning capabilities, and the support of an open-source community. Its ability to provide more nuanced and adaptable interactions makes it a superior choice for businesses and developers looking to harness the power of AI.

We invite you to share your thoughts and experiences with OpenChat and GPT-3. Have you noticed the differences in your interactions? Leave a comment below and join the conversation about the future of AI communication.

multi_agent_tomulti_model

In [ ]:
llm_config_text = {
    "seed": 42,
    "temperature": 0,
    "functions": [
        {
            "name": "generate_text",
            "description": "Generate detailed text based on a given description",
            "parameters": {
                "type": "object",
                "properties": {
                    "description": {
                        "type": "string",
                        "description": "The description for which to generate text",
                    }
                },
                "required": ["description"],
            },
        }
    ],
    "config_list": config_list,
    "timeout": 120,
}

text_agent = autogen.AssistantAgent(
    name="text_generator",
    llm_config=llm_config_text,
    system_message="""You are a text generation assistant. Generate detailed text based on the given description. Ensure the text is coherent and relevant."""
)


In [ ]:
llm_config_code = {
    "seed": 42,
    "temperature": 0,
    "functions": [
        {
            "name": "generate_code",
            "description": "Generate code snippets based on a given description",
            "parameters": {
                "type": "object",
                "properties": {
                    "description": {
                        "type": "string",
                        "description": "The description for which to generate code",
                    }
                },
                "required": ["description"],
            },
        }
    ],
    "config_list": config_list,
    "timeout": 120,
}

code_agent = autogen.AssistantAgent(
    name="code_generator",
    llm_config=llm_config_code,
    system_message="""You are a code generation assistant. Generate code snippets based on the given description. Ensure the code is correct and relevant."""
)


In [ ]:
llm_config_image = {
    "seed": 42,
    "temperature": 0,
    "functions": [
        {
            "name": "generate_image",
            "description": "Generate an image based on a given description",
            "parameters": {
                "type": "object",
                "properties": {
                    "description": {
                        "type": "string",
                        "description": "The description for which to generate an image",
                    }
                },
                "required": ["description"],
            },
        }
    ],
    "config_list": config_list,
    "timeout": 120,
}

image_agent = autogen.AssistantAgent(
    name="image_generator",
    llm_config=llm_config_image,
    system_message="""You are an image generation assistant. Generate an image based on the given description. Ensure the image is relevant and accurate."""
)


In [ ]:
def initiate_conversation_with_agents(description):
    # Khởi tạo các cuộc trò chuyện với từng agent
    response_text = user_proxy.initiate_chat(
        text_agent,
        message=f"Generate a detailed text based on: {description}"
    )
    
    response_code = user_proxy.initiate_chat(
        code_agent,
        message=f"Generate a code snippet based on: {description}"
    )
    
    response_image = user_proxy.initiate_chat(
        image_agent,
        message=f"Generate an image based on: {description}"
    )

    return response_text, response_code, response_image

# Ví dụ sử dụng
description = "an innovative AI solution"
text, code, image = initiate_conversation_with_agents(description)

print("Text Response:", text)
print("Code Response:", code)
print("Image Response:", image)
